<p> 
<hr>
<i><H5>SET UP</H5></i>
For this project we aim to use the EC2 Instance Connect End Point to connect to resources in the VPC (Virtual Private Cloud) using the AWS CLI. Once set up an IAM User will be created with programmatic access which will be configured to our computer. This will allow us to use the EC2 Instance connect endpoint to connect to resources in the private subnet and the public subnet of our VPC using AWS CLI
</p>
<br>

<H5><i>Pre-Installation & Requirements</i><H5>
Prerequisites for the project include:

<ul>
<li>Homebrew</li>
<li>iterm2</li>
<li>AWS CLI</li>
</ul>


<hr>

<ol>
<li><i>AWS - IAMS</i></li>
(Creating new Administrative Access User)
<br>
<img src="Images/UsersSelect.png" alt="Drawing" style="width: 300px;"/>

![CreateUser](Images/CreateUser.png)

Once created Attach required policies: 

![UserName](Images/UserName.png)
![AttachPolicies](Images/AttachPolicies.png)


<hr>
<li><i>Secret Access Key for IAM User</i></li>
<br>

![ClickUser](Images/ClickUser.png)
![CreateAccessKey](Images/CreateAccessKey.png)
![CLIAttribute](Images/CLIAttribute.png)
![AccessKeyShow](Images/AccessKeyShow.png)


<hr>
<li><i>Configuring the secret access Key on the AWS CLI and ECE Instance
<br>
This will be needed to utilise AWS resource in the VP
<br>
Open Terminal and enter 
<br>
<code>
> aws configure
</code>

![aws configure](Images/awsConfigure.png)
<br>
Once set up credentials can be found through the following: 
<br>
<code>
> cd
> ls -a 
</code>

<br>
Note: "-a" syntax is used to reveal hidden folders
<br>
<code>
> cd .aws
</code>

<br>
present 2 repositories "config" and "credentials", utilise <i>cat</i> command to access contents
<br>

<code>
> cat config
<br>[default]
<br>region = eu-west
</code>

<br>
Configuration has created a default where the region has been stored

<hr>
<li> Testing AWS CLI Set Up 
Enter the following:
<br> 
<code>
> aws iam get-user
</code>

![Getuser](Images/GetUser.png)
</ol>





Creating a Custom VPC 
We will be creating a 3 tier VPC containing the following 
- Tier 1: 
    Public Subnet holding resources such as
    - NAT Gateway 
    - Load Balancer 
    - Bastion Host 

- Tier 2
    Private Subnet 
    - Web servers
- Tier 3
    Another Private Subnet
    - DataBase 

We will be duplicating the subnet accross multiple availability zones for high availability and Fault Tolerance 
We will also create an Internet Gateway and Custom Route table to allow resources in the VPC to have access to the internet 
We will cover the Route Table's purpose in the VPC 

To create your VPC, you must first need to select the region where you want to create the VPC 

![Search VPC](Images/VPCSearch.png)

Ensure you have the region where you want to create your VPC

![Select Region](Images/VPCRegionSelect.png)


![Create VPC](Images/CreateVPC.png)
Name your VPC 

Enter a IPv4 CIDR of your choice 
![VPC IPv4](Images/VPCName.png)

(What is an IPv4 CIDR)
Classless Inter-Domain Routing (CIDR) notation is a way to represent an IP address network range, including IPv4 addresses. CIDR notation uses a networking routing prefix and a prefix length to represent an IP address
![IPv4Explained](Images/IPv4Explained.png)


![VPC Finalise](Images/VPCIPv4.png)

Once the VPC is created click on Actions 
![Click Actions](Images/EnableDNSHostName.png)

Click the tickbox for Enabling DNS Host Names and Save the feature to your VPC
![Save DNS HostName](Images/SaveEnable.png)

The following step is to set up a Internet Gateway 
This allows us to facilitate the connection between the resources within the VPC and the internet  
Please note that you can only attach one Internat Gateway per VPC

When a VPC is created a default Internet Gateway is created which can be viewed here: 
![Default Internet Gateway](Images/DefaultInternetGateway.png)

When you filter for the VPC just created we can see there are no existing Internet Gateways linked to the Dev VPC created
![Filter for VPC](Images/FilterVPC.png)
![Filtered Gateways](Images/FilteredGateways.png)

Note you can only attach 1 internet Gateway for 1 VPC

You can create an Iternet Gateway by clicking the following: 
![Create Internet Gateway](Images/CreateInternetGateway.png)

Once created you can attach a VPC to the selected Internet Gateway by selecting the VPC alias recently created
![Attach Internet Gateway](Images/AttachIGtoVPC.png)
![Select VPC](Images/SelectVPC.png)
![Attached Status](Images/AttachedStatus.png)






The next step is creating a subnet for the VPC. 
This will allow us to logically place AWS resources based on different needs (Private Subnets and Private Subnets)

Private Subnets we will use to add: 
- EC2 Instances 
- RDS Instance

Public Subnet:
- ALB
- NAT Gateway

We are going to start by creating a Subnet:
![](Images/CreateSubnet.png)
![](Images/Select%20DEV%20VPC.png)
![](Images/EnterSubnetDetails.png)

Using the same steps as above we will create 2 more subnets so the following applies: 
(In the same AWS Region as the VPC: eu-west-2a)
1. Public Subnet - IPv4 CIDR: 10.0.0.0/24
2. Private App Subnet - IPv4 CIDR: 10.0.2.0/24
3. Private Data Subnet - IPv4 CIDR: 10.0.4.0/24

We are going to have a mirror image of these subnets in a second availability zone (eu-west-2b) with the following properties: 
1. Public Subnet - IPv4 CIDR: 10.0.1.0/24
2. Private App Subnet - IPv4 CIDR: 10.0.3.0/24
3. Private Data Subnet - IPv4 CIDR: 10.0.5.0/24

Ensure all subnets are located in the same availability zone

Next we will enable auto assign on the Public subnets
![](Images/EnableAutoAssign.png)

This ensures AWS automatically assigns a Public IPv4 to any resources launched in these subnets, this allows AWS to automatically assign a public IPv4 address to any resource launced in the subnet. 


Creating a Route Table 

To make a subnet public: 
1. Create a route table (containing a route to the internet)
2. Subnets then associated with the route table will become public

![](Images/CreateRouteTable.png)

Note: When a VPC is created AWS automatically creates a main route table (as the default route table in that VPC)

In the creation of a route table you can select a VPC which will be used with the Route table 
![](Images/SelectVPCForRouteTable.png)

Our next step is to add a route to the route table

Route are specified under the routes section of the selected Route Table. 
In the below example we have a Target set to "local" which means it will only be routing traffic within the VPC. 
![](Images/DefaultRoute.png)

In order to create a route to the Internet we must first edit route 
![](Images/EditRoute.png)

Note: In networking any time we want traffic to have access to the internet the destination becomes 0.0.0.0/0 
Under target click Internet Gateway and select the gateway associated with the Dev Internet Gateway
![](Images/RouteToInternet.png)

Now that a route has been addedd to the route table to route traffic to the internet, any subnet associated with the specific route table will become public. So in order to associate the Public subnets with the route table : 
- Select Subnet associations 

![](Images/EditSubnetAssociation.png)

- Edit Subnet associations

![](Images/SelectPublicSubnets.png)



Main Takeways: 
To make a subnet public you have to create a route table containg a route that is routing traffic to the internet (0.0.0.0/0) once created you associate the route table with the subnets. 

